<h3>Total Costs Regression Model Building and Exploration</h3><br />
This notebook shows the process of building regression models for total costs. It shows the data distribution, noramlization process, baseline performance of different regression models, a hypertuned version of the best performing model, and performance on a normalized dataset.

In [ ]:
import sys
import platform
plat = platform.system()
if plat == 'Windows':
    sys.path.insert(0, '..\src\helpers')
elif plat =='Linux' or plat=='Darwin':
    sys.path.insert(0, '../src/helpers')

In [ ]:
from data_cleaners import *
from model_building_helpers import *
import pandas as pd
import numpy as np
from sklearn import preprocessing, svm
from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics
from sklearn.linear_model import LinearRegression
from sklearn.compose import TransformedTargetRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.linear_model import Ridge, Lasso
from sklearn.model_selection import cross_val_score, RandomizedSearchCV, StratifiedKFold, cross_validate
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

In [ ]:
data_file_path = '../data'
all_patient_df = load_data('all', data_file_path)
all_patient_df = all_patient_df[all_patient_df['Total Costs'] > 0]

In [ ]:
#Plot distibution of length of stay target variable
sns.histplot(all_patient_df['Total Costs'], bins=50, palette='orangered')
current_values_y = plt.gca().get_yticks()
current_values_x = plt.gca().get_xticks()
plt.gca().set_yticklabels(['{:,.0f}'.format(x) for x in current_values_y])
plt.gca().set_xticklabels(['{:,.0f}'.format(x) for x in current_values_x])
plt.xlabel('Total Cost', fontsize=14, fontweight='bold')
plt.ylabel('Count', fontsize=14, fontweight='bold')
plt.title('Distribution of Total Cost Variable', fontsize=17, fontweight='bold')
plt.tight_layout(rect = [0, 0, 1, 0.988])

In [ ]:
#Plot distibution of length of stay target variable
fig, axes = plt.subplots(2, 2, figsize=(18, 8), sharey=False)
fig.subplots_adjust(hspace=0.45, wspace=0.25)
fig.suptitle('Total Costs Distribution After Normalization', fontsize=17, fontweight='bold')

sns.histplot(ax=axes[0, 0], data=np.log(all_patient_df['Total Costs']), bins=10, palette='skyblue')
axes[0, 0].set_title('Log Transform', fontweight='bold', fontsize=14)
axes[0, 0].set_xlabel('Total Costs', fontweight='bold', fontsize=12)
axes[0, 0].set_ylabel('Count', fontweight='bold', fontsize=12)
axes[0, 0].tick_params(axis='x', size=12)

sns.histplot(ax=axes[0, 1], data=np.exp(all_patient_df['Total Costs']), bins=10, palette='skyblue')
axes[0, 1].set_title('Power Transform', fontweight='bold', fontsize=14)
axes[0, 1].set_xlabel('Total Costs', fontweight='bold', fontsize=12)
axes[0, 1].set_ylabel('Count', fontweight='bold', fontsize=12)
axes[0, 1].tick_params(axis='x', size=12)

sns.histplot(ax=axes[1, 0], data=np.sqrt(all_patient_df['Total Costs']), bins=10, palette='skyblue')
axes[1, 0].set_title('Square Root Transform', fontweight='bold', fontsize=14)
axes[1, 0].set_xlabel('Total Costs', fontweight='bold', fontsize=12)
axes[1, 0].set_ylabel('Count', fontweight='bold', fontsize=12)
axes[1, 0].tick_params(axis='x', size=12)

sns.histplot(ax=axes[1, 1], data=stats.boxcox(all_patient_df['Total Costs'], 0), bins=10, palette='skyblue')
axes[1, 1].set_title('Box Cox Transform', fontweight='bold', fontsize=14)
axes[1, 1].set_xlabel('Total Costs', fontweight='bold', fontsize=12)
axes[1, 1].set_xlabel('Length of Stay', fontweight='bold', fontsize=12)
axes[1, 1].set_ylabel('Count', fontweight='bold', fontsize=12)
axes[1, 1].tick_params(axis='x', size=12)

In [ ]:
#Since training all the data takes a long time, take a stratified sample of 
# the data for initial baseline model performance testing
if len(all_patient_df) > 30000:
    all_patient_stratified_sample_df = all_patient_df.groupby('Total Costs', group_keys=False).apply(
    lambda x: x.sample(int(np.rint(30000*len(x)/len(all_patient_df))))).sample(frac=1).reset_index(drop=True)
else:
    all_patient_stratified_sample_df = all_patient_df.copy()

X = all_patient_stratified_sample_df.loc[:, ~all_patient_stratified_sample_df.columns.isin(['Length of Stay', 'Total Costs', 'Total Charges', 'Patient Disposition'])]
y = all_patient_stratified_sample_df[['Total Costs']]

categorical_columns = list(all_patient_stratified_sample_df.select_dtypes(include='object'))

X = pd.get_dummies(X)

X = X.loc[:, ~X.columns.isin(categorical_columns)]
y['Total Costs'] = y['Total Costs'].astype('float64')


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25)

In [ ]:
#Train baseline models (default parameters) on unnormalized data
models =  {
    'RidgeRegression': Ridge(random_state=42),
    'Lasso': Lasso(random_state=42),
    'GradientBoostedRegressor': GradientBoostingRegressor(random_state=42),
    'RandomForestRegressor': RandomForestRegressor(random_state=42, n_estimators = 10) #Set this parameter so the model can run faster
}

model_scores = []


scoring = {'r2': 'r2'}

for model_key in models.keys():
    print(model_key)
    kf = StratifiedKFold(n_splits=6)
    # Perform cross-validation
    cv_scores = cross_validate(models[model_key], X, np.ravel(y), cv=kf, scoring=scoring)
    # Append the results
    model_scores.append({'Model' : model_key,
        'all_r2_scores': cv_scores['test_r2'],
        'avg_r2_score': np.mean(cv_scores['test_r2'])
    })

In [ ]:
scores_df = pd.DataFrame(data=model_scores)

#Plot distibution of length of stay target variable
fig, axes = plt.subplots(1, 1, figsize=(18, 8), sharey=False)
fig.subplots_adjust(hspace=0.45, wspace=0.25)
fig.suptitle('Evaluation Scores Total Cost Prediction For Regression Models', fontsize=17, fontweight='bold')


barplot = sns.barplot(x=scores_df['Model'], y=scores_df['avg_r2_score'], palette=['#5CED73'])
plt.ylabel('Average R2 Score', fontweight='bold', fontsize=14)
plt.xlabel('Model Type', fontweight='bold', fontsize=14)
plt.xticks(ticks= [0 ,1, 2, 3], labels=['Ridge\nRegeression', 'Lasso\nRegeression', 'GradientBoosted\nRegression', 'RandomForest\nRegression'], size=14)
plt.bar_label(barplot.containers[0], fmt='%.2f', padding=2, fontsize=14)
plt.yticks(fontsize=14)

In [ ]:
#Train models on normalized data
from sklearn.model_selection import cross_validate
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
from sklearn.preprocessing import PowerTransformer

#Train baseline models (default parameters) on unnormalized data
models =  {
    'RidgeRegression': Ridge(),
    'GradientBoostedRegressor': GradientBoostingRegressor()
}

model_scores_transformed = []


scoring = {'r2': 'r2',
           'mae': 'neg_mean_absolute_error',
           'mse': 'neg_mean_squared_error'}

for model_key in models.keys():
    print(model_key)
    kf = StratifiedKFold(n_splits=6)
    # Perform cross-validation
    regr_trans = TransformedTargetRegressor(regressor=models[model_key],
                                        transformer=PowerTransformer(method='box-cox'))
    cv_scores = cross_validate(regr_trans, X, y, cv=kf, scoring=scoring)
    # Append the results
    model_scores_transformed.append({'Model' : model_key,
        'Transformation': 'Box-Cox',
        'all_r2_scores': cv_scores['test_r2'],
        'avg_r2_score': np.mean(cv_scores['test_r2']),
        'all_mae_scores': cv_scores['test_mae'],
        'avg_mae_score': np.mean(cv_scores['test_mae']),
        'all_mse_scores': cv_scores['test_mse'],
        'avg_mse_score': np.mean(cv_scores['test_mse'])
    })

for model_key in models.keys():
    print(model_key)
    kf = StratifiedKFold(n_splits=6)
    # Perform cross-validation
    regr_trans = TransformedTargetRegressor(regressor=models[model_key],
                                        func=np.log, inverse_func=np.exp)
    cv_scores = cross_validate(regr_trans, X, y, cv=kf, scoring=scoring)
    # Append the results
    model_scores_transformed.append({'Model' : model_key,
        'Transformation': 'Log Transform',
        'all_r2_scores': cv_scores['test_r2'],
        'avg_r2_score': np.mean(cv_scores['test_r2']),
        'all_mae_scores': cv_scores['test_mae'],
        'avg_mae_score': np.mean(cv_scores['test_mae']),
        'all_mse_scores': cv_scores['test_mse'],
        'avg_mse_score': np.mean(cv_scores['test_mse'])
    })

for model_key in models.keys():
    print(model_key)
    kf = StratifiedKFold(n_splits=6)
    # Perform cross-validation
    regr_trans = TransformedTargetRegressor(regressor=models[model_key],
                                        func=np.sqrt, inverse_func=np.square)
    cv_scores = cross_validate(regr_trans, X, y, cv=kf, scoring=scoring)
    # Append the results
    model_scores_transformed.append({'Model' : model_key,
        'Transformation': 'Square Root',
        'all_r2_scores': cv_scores['test_r2'],
        'avg_r2_score': np.mean(cv_scores['test_r2']),
        'all_mae_scores': cv_scores['test_mae'],
        'avg_mae_score': np.mean(cv_scores['test_mae']),
        'all_mse_scores': cv_scores['test_mse'],
        'avg_mse_score': np.mean(cv_scores['test_mse'])
    })

In [ ]:
model_scores_transformed_df = pd.DataFrame(data=model_scores_transformed)

log_transform_df = model_scores_transformed_df[model_scores_transformed_df['Transformation'] == 'Log Transform']
sqrt_df = model_scores_transformed_df[model_scores_transformed_df['Transformation'] == 'Square Root']
box_cox_df = model_scores_transformed_df[model_scores_transformed_df['Transformation'] == 'Box-Cox']

color_values = ['skyblue', '#5CED73']

fig, axes = plt.subplots(1, 3, figsize=(18, 5), sharey=False)
fig.subplots_adjust(hspace=0.95, wspace=0.50)
fig.suptitle('Model Performace by Transformation Type', fontsize=17, fontweight='bold')

sns.barplot(ax=axes[0], x=log_transform_df['Model'], y=np.absolute(log_transform_df['avg_r2_score']), palette = color_values)
axes[0].set_title('Log Transform Average R2 Score', fontweight='bold', fontsize=16)
axes[0].set_ylabel('Average R2 Score', fontweight='bold', fontsize=16)
axes[0].set_xlabel('Model Type', fontweight='bold', fontsize=16)
axes[0].set_xticklabels(labels=['Ridge\nRegeression', 'GradientBoosted\nRegression'], fontsize=16)
axes[0].set_ylim(0, 0.5)
axes[0].bar_label(axes[0].containers[0], fmt='%.2f', padding=2, fontsize=16)
current_values_y = axes[0].get_yticks()
axes[0].set_yticklabels([round(x, 2) for x in current_values_y], fontsize=16)

sns.barplot(ax=axes[1], x=sqrt_df['Model'], y=np.absolute(sqrt_df['avg_r2_score']), palette = color_values)
axes[1].set_title('Square Root Transform Average R2 Score', fontweight='bold', fontsize=16)
axes[1].set_ylabel('Average R2 Score', fontweight='bold', fontsize=16)
axes[1].set_xlabel('Model Type', fontweight='bold', fontsize=16)
axes[1].set_xticklabels(labels=['Ridge\nRegeression', 'GradientBoosted\nRegression'], fontsize=16)
axes[1].set_ylim(0, 0.5)
axes[1].bar_label(axes[1].containers[0], fmt='%.2f', padding=2, fontsize=16)
current_values_y = axes[1].get_yticks()
axes[1].set_yticklabels([round(x, 2) for x in current_values_y], fontsize=16)

sns.barplot(ax=axes[2], x=box_cox_df['Model'], y=np.absolute(box_cox_df['avg_r2_score']), palette = color_values)
axes[2].set_title('Box Cox Transformation Average R2 Score', fontweight='bold', fontsize=16)
axes[2].set_ylabel('Average R2 Score', fontweight='bold', fontsize=16)
axes[2].set_xlabel('Model Type', fontweight='bold', fontsize=16)
axes[2].set_xticklabels(labels=['Ridge\nRegeression', 'GradientBoosted\nRegression'], fontsize=16)
axes[2].set_ylim(0, 0.5)
axes[2].bar_label(axes[2].containers[0], fmt='%.2f', padding=2, fontsize=16)
current_values_y = axes[2].get_yticks()
axes[2].set_yticklabels([round(x, 2) for x in current_values_y], fontsize=16)

plt.tight_layout(rect = [0, 0, 1, 0.988])




In [ ]:
params = {
    'regressor__n_estimators': [10, 50, 100, 500],
    'regressor__learning_rate': [0.0001, 0.001, 0.01, 0.1, 1.0],
    'regressor__subsample': [0.5, 0.7, 1.0],
    'regressor__max_depth': [1, 3, 7, 9, 10]
}

grid_search = RandomizedSearchCV(estimator = TransformedTargetRegressor(regressor=GradientBoostingRegressor(random_state=42), func=np.sqrt, inverse_func=np.square), param_distributions=params, 
                      cv = StratifiedKFold(n_splits=3), n_iter=20, n_jobs = -1, verbose = 2, scoring='r2')

grid_search.fit(X_train, y_train)

print(grid_search.best_params_)
print(grid_search.best_score_)

In [ ]:
X = all_patient_df.loc[:, ~all_patient_df.columns.isin(['Length of Stay', 'Total Costs', 'Total Charges', 'Patient Disposition'])]
y = all_patient_df[['Total Costs']]

categorical_columns = list(all_patient_df.select_dtypes(include='object'))

X = pd.get_dummies(X)

X = X.loc[:, ~X.columns.isin(categorical_columns)]
y['Total Costs'] = y['Total Costs'].astype('float64')


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25)
regr_trans = TransformedTargetRegressor(regressor=GradientBoostingRegressor(learning_rate = 0.01, 
                                                                            max_depth = 9, 
                                                                            n_estimators = 500, 
                                                                            subsample = 0.7),
                                        func=np.sqrt, inverse_func=np.square)
regr_trans.fit(X_train, y_train)
y_pred = regr_trans.predict(X_test)
print('=================================================================================================================')
print("R2 Score = ", r2_score(y_test, y_pred), "Mean Absolute Error (MAE) = ", mean_absolute_error(y_test, y_pred), "Mean Squared Error (MSE) = ", mean_squared_error(y_test, y_pred))
print('=================================================================================================================')

In [ ]:
data={'feature_names':regr_trans.regressor_.feature_names_in_,'feature_importance':regr_trans.regressor_.feature_importances_}
features_importance_df = pd.DataFrame(data)
features_importance_df.sort_values(by=['feature_importance'], ascending=False,inplace=True)
features_importance_df.head()
feature_importance_to_10_df = features_importance_df[0:10]

In [ ]:
plt.figure(figsize=(15,15))

sns_barplot = sns.barplot(x=feature_importance_to_10_df['feature_importance'], y=feature_importance_to_10_df['feature_names'], palette = ['skyblue'])
plt.xlabel('Feature Importance', fontsize=14, fontweight='bold')
plt.ylabel('Feature Name', fontsize=14, fontweight='bold')
plt.title('Top 10 Feature Importances For Gradient Boosted Regressor\n(Total Cost)', fontsize=17, fontweight='bold')
plt.tick_params(axis='both', which='major', labelsize=12)
plt.bar_label(sns_barplot.containers[0], fmt='%.2f', padding=1, fontsize=12)